In [ ]:
from hpsklearn import HyperoptEstimator, sgd
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

space = {
    'max_depth' : hp.choice('max_depth', range(2, 20, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.3, 0.01),
    'min_child_weight' : hp.quniform('min_child_weight', 5, 30, 1),
    'subsample' : hp.quniform('subsample', 0.25, 0.4, 0.05)
}

In [ ]:
def objective(space):
    from sklearn.model_selection import cross_val_score
    
    xgboost = xgb.XGBClassifier(
        objective='binary:logistic', use_label_encoder=False, eval_metric='logloss',
        scale_pos_weight=0.2,
        max_depth = int(space['max_depth']),
        learning_rate = space['learning_rate'],
        min_child_weight = space['min_child_weight'],
        subsample = space['subsample']
    )
    
    auc = cross_val_score(xgboost, X_train_nf, y_train, cv=10, scoring="roc_auc").mean()

    return{'loss': 1-auc, 'status': STATUS_OK }

In [ ]:
best = fmin(
    fn=objective,
    space=space,       
    algo=tpe.suggest,       
    max_evals=30
)

In [ ]:
xgb_hpr = xgb.XGBClassifier(
    objective='binary:logistic', use_label_encoder=False, eval_metric='logloss',
    scale_pos_weight=0.2,
    max_depth = best['max_depth'],
    learning_rate = best['learning_rate'],
    min_child_weight = best['min_child_weight'],
    subsample = best['subsample']
).fit(X_train_nf, y_train)

y_pred = xgb_hpr.predict_proba(X_val_nf)[:,1]
eval_metrics(y_pred, y_val, praba_threshold=0.5)